In [ ]:
## Watsonx.data Credentials
UsernameCPD = 'ibmlhapikey' #The username for your watsonx.data instance. Here, ibmlhapikey.
PasswordCPD = '****' #The API key generated for the watsonx.data

##Cos bucket credentials
LH_S3_ACCESS_KEY = "****"
LH_S3_SECRET_KEY = "****" 

#DB2 details
db_host_name = 'jdbc:db2://<DBHOSTNAME>'
db_user_name='****'
db_password='****'
#####required

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F


spark = SparkSession.builder.appName('sparky').getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")
conf = sc.getConf()

conf.set("spark.hive.metastore.uris", f"thrift://*WXDENDPOINT*") 
conf.set("spark.hive.metastore.use.SSL", "true")
conf.set("spark.hive.metastore.truststore.type", "JKS")
conf.set("spark.hive.metastore.truststore.path", "file:///opt/ibm/jdk/lib/security/cacerts")
conf.set("spark.hive.metastore.truststore.password", "changeit")

conf.set("spark.hive.metastore.client.auth.mode", "PLAIN")
conf.set("spark.hive.metastore.client.plain.username", UsernameCPD)
conf.set("spark.hive.metastore.client.plain.password", PasswordCPD)

conf.set("fs.s3a.access.key", LH_S3_ACCESS_KEY)
conf.set("fs.s3a.secret.key", LH_S3_SECRET_KEY)
conf.set("fs.s3a.endpoint", f"http://s3.us-south.cloud-object-storage.appdomain.cloud")

conf.set("fs.s3a.connection.ssl.enabled",  "true")
conf.set("fs.s3a.path.style.access",       "true")
conf.set("fs.s3a.connection.ssl.enabled",  "true")
conf.set("fs.s3a.impl",                    "org.apache.hadoop.fs.s3a.S3AFileSystem")
conf.set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')


conf.set("spark.jars.packages", "io.delta:delta-core_2.12:2.2.0")
conf.set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
conf.set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

conf.set("spark.sql.catalog.db2catalog.type", "jdbc")  # JDBC catalog type for Db2
conf.set("spark.sql.catalog.db2catalog.jdbc.url", db_host_name)  # JDBC URL for Db2
conf.set("spark.sql.catalog.db2catalog.jdbc.user", db_user_name)  # Username for Db2
conf.set("spark.sql.catalog.db2catalog.jdbc.password", db_password)  # Password for Db2
conf.set("spark.sql.catalog.db2catalog.jdbc.sslConnection", "true")  # Enable SSL for JDBC connection
conf.set("spark.sql.catalog.db2catalog.jdbc.sslTrustStoreLocation", "s3a://spark-analytics-target/truststore.jks")  # Path to the truststore file if it is ssl enabled
conf.set("spark.sql.catalog.db2catalog.jdbc.sslTrustStorePassword", "*password*")  # Password for the truststore


sc.stop()

spark = (SparkSession
         .builder.config(conf=conf)
         .enableHiveSupport()
         .getOrCreate()
        )

sc = spark.sparkContext
conf = sc.getConf().getAll()

for config in conf:
    print(config)


#####required

('spark.hive.metastore.truststore.password', 'changeit')
('spark.hive.metastore.truststore.type', 'JKS')
('spark.sql.catalog.db2catalog.jdbc.sslTrustStorePassword', '*****')
('spark.network.crypto.keyLength', '256')
('spark.driver.memory', '3600M')
('spark.app.id', 'app-20241202130549-0001')
('spark.network.crypto.enabled', 'true')
('spark.hive.metastore.truststore.path', 'file:///opt/ibm/jdk/lib/security/cacerts')
('spark.authenticate.enableSaslEncryption', 'true')
('spark.ui.killEnabled', 'false')
('spark.driver.host', 'spark-master-headless-bb3bfc7f-a72f-45e5-b047-d902e736dec6')
('spark.app.submitTime', '1733144677576')
('spark.r.command', '/home/spark/shared/R/bin/Rscript')
('spark.hadoop.fs.stocator.glob.bracket.support', 'true')
('spark.local.dir', '/tmp/spark/scratch')
('spark.authenticate.secret', 'a1890ed2-bb98-479b-b127-ea9c295f44e6')
('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')
('spark.ui.reverseProxy', 'false')
('spark.executor.memory', '3600M')
('spark.driver.

In [3]:
def create_databases(spark):
    # Base S3 location
    base_s3_path = "s3a://spark-dl/"
    
    # Databases to be created with corresponding paths
    databases = {
        "bronze_ec": f"{base_s3_path}bronze_ec/",
        "silver_ec": f"{base_s3_path}silver_ec/",
        "gold_ec": f"{base_s3_path}gold_ec/"
    }

    # Loop through each database and create it if not exists
    for db_name, db_location in databases.items():
        try:
            spark.sql(f"CREATE DATABASE IF NOT EXISTS spark_catalog.{db_name} LOCATION '{db_location}'")
            print(f"Database {db_name} created at {db_location}")
        except Exception as e:
            print(f"Error creating database {db_name} at {db_location}: {e}")

# Example usage:
create_databases(spark)

#####required

Database bronze_ec created at s3a://spark-dl/bronze_ec/
Database silver_ec created at s3a://spark-dl/silver_ec/
Database gold_ec created at s3a://spark-dl/gold_ec/


In [4]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

def recommend_products_and_accessories_per_customer(spark):
    # Load real-time and historical sales data
    realtime_sales = spark.read.format("delta").load("s3a://spark-dl/bronze_ec/realtime_sales_new/")
    historical_sales = spark.read.format("parquet").load("s3a://sparkdb2-target/bronze_hist_sales/data/")
    
    # Cast relevant columns and merge data
    realtime_sales_selected = realtime_sales.select(
        F.col("sales_id").cast("long"),
        F.col("product_id").cast("long"),
        F.col("accessory_id").cast("long"),
        F.col("customer_id").cast("long"),
        F.col("quantity").cast("long"),
        F.col("total_amount").cast("double")
    )
    historical_sales_selected = historical_sales.select(
        F.col("SALES_ID").cast("long"),
        F.col("PRODUCT_ID").cast("long"),
        F.col("ACCESSORY_ID").cast("long"),
        F.col("CUSTOMER_ID").cast("long"),
        F.col("QUANTITY").cast("long"),
        F.col("TOTAL_AMOUNT").cast("double")
    )
    
    # Combine real-time and historical sales data
    combined_sales = realtime_sales_selected.union(historical_sales_selected)
    
    # Step 1: Get each customer's purchase history (both products and accessories)
    customer_purchase_data = combined_sales.select("customer_id", "product_id", "accessory_id", "quantity", "total_amount")
    
    # Step 2: Calculate Average Price for Each Customer per Product
    customer_avg_price_df = customer_purchase_data.groupBy("customer_id", "product_id").agg(
        F.sum("total_amount").alias("total_spent"),
        F.sum("quantity").alias("total_quantity")
    )
    
    # Calculate average price spent by each customer per product
    customer_avg_price_df = customer_avg_price_df.withColumn(
        "avg_price", F.col("total_spent") / F.col("total_quantity")
    )
    
    # Step 3: Identify the most frequent brands purchased by each customer
    customer_brand_preference = customer_purchase_data.join(
        spark.read.format("jdbc") \
            .option("url", db_host_name) \
            .option("user", db_user_name) \
            .option("password", db_password) \
            .option("dbtable", "ECOMMERCE.PRODUCTS") \
            .option("sslConnection", "true") \
            .load(),
        "product_id"
    ).groupBy("customer_id", "brand").agg(
        F.count("*").alias("purchase_count")
    )
    
    # Rename 'brand' columns to avoid ambiguity
    customer_brand_preference = customer_brand_preference.withColumnRenamed("brand", "customer_brand")
    
    # Step 4: Rank brands for each customer based on purchase frequency
    customer_brand_preference = customer_brand_preference.withColumn(
        "rank", F.row_number().over(
            Window.partitionBy("customer_id").orderBy(F.desc("purchase_count"))
        )
    ).filter(F.col("rank") == 1)  # Keep only the top brand
    
    # Step 5: Filter products based on price range and brand preference
    recommendations = customer_avg_price_df.join(
        customer_brand_preference,
        "customer_id",
        "left_outer"  # This will handle first-time buyers who have no brand preference
    ).join(
        spark.read.format("jdbc") \
            .option("url", db_host_name) \
            .option("user", db_user_name) \
            .option("password", db_password) \
            .option("dbtable", "ECOMMERCE.PRODUCTS") \
            .option("sslConnection", "true") \
            .load(),
        "product_id"
    ).withColumnRenamed("brand", "product_brand")  # Rename the 'brand' column in products table
    
    # Step 6: Handle first-time buyers with no brand preference (left join)
    recommendations = recommendations.withColumn(
        "product_brand",
        F.coalesce(F.col("customer_brand"), F.col("product_brand"))  # If no brand preference, use product's brand
    )
    
    # Step 7: Filter by price range for first-time buyers or those with a preference
    recommendations = recommendations.filter(
        (F.col("avg_price") * 0.8 <= F.col("PRICE")) &
        (F.col("avg_price") * 1.2 >= F.col("PRICE"))
    ).select(
        "customer_id", "product_id", "PRODUCT_NAME", "PRICE", "product_brand"
    )
    
    # For first-time buyers, if there's no previous data, recommend popular products in general
    first_time_buyers = recommendations.filter(F.col("customer_id").isNull())
    
    if first_time_buyers.count() > 0:
        # For first-time buyers, recommend popular products based on product sales
        popular_products = spark.read.format("jdbc") \
            .option("url", db_host_name) \
            .option("user", db_user_name) \
            .option("password", db_password) \
            .option("dbtable", "ECOMMERCE.PRODUCTS") \
            .option("sslConnection", "true") \
            .load()
        
        # Add logic here to select popular products based on sales volume or customer ratings
        # For example, top-selling products or those with the highest ratings
        popular_products = popular_products.orderBy(F.desc("SALES_COUNT")).limit(10)
        
        recommendations = recommendations.union(popular_products)  # Combine popular products with other recommendations

    # Step 8: Display final recommendations
    recommendations.show(truncate=False)
    
    # Optionally, save recommendations to a table for each customer
    recommendations.write \
        .mode("overwrite") \
        .format("delta") \
        .saveAsTable("gold_ec.customer_product_recommendations")
    
    print("Customer-specific recommendations table created.")
    
    # Call the recommendation function
recommend_products_and_accessories_per_customer(spark)


+-----------+----------+-----------------------------------------+------+-------------+
|customer_id|product_id|PRODUCT_NAME                             |PRICE |product_brand|
+-----------+----------+-----------------------------------------+------+-------------+
|307        |201       |REDMI Note 12 Pro 5G (Onyx Black, 128 GB)|299.99|Xiaomi       |
|301        |201       |REDMI Note 12 Pro 5G (Onyx Black, 128 GB)|299.99|Xiaomi       |
|306        |201       |REDMI Note 12 Pro 5G (Onyx Black, 128 GB)|299.99|Xiaomi       |
|309        |201       |REDMI Note 12 Pro 5G (Onyx Black, 128 GB)|299.99|Xiaomi       |
|303        |201       |REDMI Note 12 Pro 5G (Onyx Black, 128 GB)|299.99|Xiaomi       |
|308        |201       |REDMI Note 12 Pro 5G (Onyx Black, 128 GB)|299.99|Xiaomi       |
|305        |201       |REDMI Note 12 Pro 5G (Onyx Black, 128 GB)|299.99|Infinix      |
|302        |201       |REDMI Note 12 Pro 5G (Onyx Black, 128 GB)|299.99|Xiaomi       |
|310        |201       |REDMI No

In [5]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

def product_Sales_Data(spark):
    # Load real-time sales data
    realtime_sales = spark.read.format("delta").load("s3a://spark-dl/bronze_ec/realtime_sales_new/")
    print("Real-time sales data loaded")

    # Load historical sales data
    historical_sales = spark.read.format("parquet").load("s3a://sparkdb2-target/bronze_hist_sales/data/")
    print("Historical sales data loaded")

    # Load promotion data
    promotion_df = spark.read.format("parquet").load("s3a://sparkdb2-target/bronze_promotions_sale/data/")
    print("Promotion data loaded")

    product_df = spark.read.format("jdbc") \
        .option("url", db_host_name) \
        .option("user", db_user_name) \
        .option("password", db_password) \
        .option("dbtable", "ecommerce.products") \
        .option("sslConnection", "true") \
        .load()

    product_df.createOrReplaceTempView("product_data")  # Create a temporary view for product data
    print("Product data loaded from JDBC")

    # Select and cast columns for real-time and historical sales
    realtime_sales_selected = realtime_sales.select(
        F.col("sales_id").cast("long").alias("sales_id"),
        F.col("product_id").cast("long").alias("product_id"),
        F.col("quantity").cast("long").alias("quantity"),
        F.col("total_amount").cast("double").alias("total_amount"),
        F.col("sale_date").cast("timestamp").alias("sale_date")
    )

    historical_sales_selected = historical_sales.select(
        F.col("SALES_ID").cast("long").alias("sales_id"),
        F.col("PRODUCT_ID").cast("long").alias("product_id"),
        F.col("QUANTITY").cast("long").alias("quantity"),
        F.col("TOTAL_AMOUNT").cast("double").alias("total_amount"),
        F.col("SALE_DATE").cast("timestamp").alias("sale_date")
    )

    # Select promotion columns
    promotion_selected = promotion_df.select(
        F.col("Promotion_ID"),
        F.col("Product_ID").alias("promotion_product_id"),
        F.col("Discount_Percentage").cast("double").alias("Discount_Percentage"),
        F.col("Promotion_Start_Date"),
        F.col("Promotion_End_Date"),
        F.col("Promotion_Type")
    )

    # Combine real-time and historical sales
    combined_sales = realtime_sales_selected.union(historical_sales_selected)

    # Add is_promotion_active column
    combined_sales_with_promo = combined_sales.join(
        promotion_selected,
        (combined_sales.product_id == promotion_selected.promotion_product_id) &  
        (combined_sales.sale_date.between(promotion_selected.Promotion_Start_Date, promotion_selected.Promotion_End_Date)),
        "left"
    ).withColumn(
        "is_promotion_active", 
        F.when(F.col("Discount_Percentage").isNotNull(), 1).otherwise(0)
    )

    # Handle nulls in sales and promotion features
    combined_sales_with_promo = combined_sales_with_promo.na.fill(0.0, subset=['total_amount', 'quantity', 'Discount_Percentage'])

    # Create a temporary view for combined sales with promotions
    combined_sales_with_promo.createOrReplaceTempView("combined_sales_with_promo")

    # Now left join with product data to include all products
    combined_sales_with_products = product_df.join(
        combined_sales_with_promo,
        product_df["PRODUCT_ID"] == combined_sales_with_promo["product_id"],
        "left"
    ).select(
        product_df["PRODUCT_ID"],              # Product ID
        product_df["PRODUCT_NAME"],            # Product Name
        F.date_trunc('month', F.col('sale_date')).alias('sale_month'),  # Sale Month
        F.coalesce(combined_sales_with_promo.total_amount, F.lit(0)).alias('total_sales'),  # Total Sales
        F.coalesce(combined_sales_with_promo.quantity, F.lit(0)).alias('total_quantity'),   # Total Quantity
        F.coalesce(combined_sales_with_promo.Discount_Percentage, F.lit(0)).alias('avg_discount'),  # Average Discount
        F.coalesce(combined_sales_with_promo.is_promotion_active, F.lit(0)).alias('promotion_active')  # Promotion Active
    )

    # Aggregate to remove duplicates by summing total_sales and total_quantity, averaging the discount, and keeping the max promotion_active flag
    combined_sales_with_products_agg = combined_sales_with_products.groupBy(
        "PRODUCT_ID", 
        "PRODUCT_NAME", 
        "sale_month"
    ).agg(
        F.sum("total_sales").alias("total_sales"),
        F.sum("total_quantity").alias("total_quantity"),
        F.avg("avg_discount").alias("avg_discount"),
        F.max("promotion_active").alias("promotion_active")  # 1 if active in any row, otherwise 0
    )

    # Write the aggregated output to a Delta table in the Gold layer
    combined_sales_with_products_agg.write \
        .mode("overwrite") \
        .format("delta") \
        .saveAsTable("gold_ec.product_sales_data")
    
    combined_sales_with_products_agg.show(truncate=False)

    print("Aggregated product sales data table created in Gold layer.")

product_Sales_Data(spark)


Real-time sales data loaded
Historical sales data loaded
Promotion data loaded
Product data loaded from JDBC
+----------+-------------------------------------------------------+-------------------+-----------------+--------------+------------+----------------+
|PRODUCT_ID|PRODUCT_NAME                                           |sale_month         |total_sales      |total_quantity|avg_discount|promotion_active|
+----------+-------------------------------------------------------+-------------------+-----------------+--------------+------------+----------------+
|223       |vivo X80 Pro (Cosmic Black, 256 GB)                    |2024-11-01 00:00:00|3999.95          |5             |0.0         |0               |
|219       |Xiaomi 11T Pro 5G Hyperphone (Celestial Magic, 256 GB) |2024-12-01 00:00:00|8166.120000000002|27            |0.0         |0               |
|210       |SAMSUNG Galaxy A04 (Green, 128 GB)                     |2024-05-01 00:00:00|999.95           |5             |0.0       

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

def process_promotions(spark):
    print("Loading customer segmentation data from Silver layer...")
    customer_segmentation = spark.read \
        .format("parquet") \
        .load("s3a://sparkdb2-target/silver_ecom/customer_segmentation/data/")

    print("Loading real-time sales data from Bronze layer...")
    real_time_sales = spark.read \
        .format("delta") \
        .load("s3a://spark-dl/bronze_ec/realtime_sales_new/")

    print("Loading product data from DB2...")
    product_data = spark.read.format("jdbc") \
        .option("url", db_host_name) \
        .option("user", db_user_name) \
        .option("password", db_password) \
        .option("dbtable", "ecommerce.products") \
        .option("sslConnection", "true") \
        .load()

    def generate_promotional_messages(segment_df, message_template):
        print(f"Generating promotional messages for segment with message: '{message_template}'")
        messages = []
        for row in segment_df.collect():
            messages.append((row['CUSTOMER_ID'], row['CUSTOMER_NAME'], message_template))
        return messages

    promotional_messages = []

    def process_customer_segment(segment_name, discount_message):
        print(f"Processing {segment_name} Customers...")
        segment_customers = customer_segmentation.filter(customer_segmentation['SEGMENT'] == segment_name)
        purchase_history = segment_customers.join(real_time_sales, segment_customers['CUSTOMER_ID'] == real_time_sales['customer_id'])
        top_products = purchase_history.groupBy('product_id').agg(
            F.sum('total_amount').alias('total_spent')
        ).orderBy('total_spent', ascending=False).limit(10)

        if not top_products.isEmpty():
            promotional_messages.extend(generate_promotional_messages(segment_customers, discount_message))
            print(f"{segment_name} Customers promotion generated.")

    # Process promotions for each segment
    process_customer_segment("High Value", "Enjoy 20% off on our premium range of products!")
    process_customer_segment("Medium Value", "Get a special 10% discount on your next purchase!")
    process_customer_segment("Low Value", "Welcome back! Enjoy 5% off your next order!")

    print("Creating DataFrame for promotional messages...")
    promotional_messages_df = spark.createDataFrame(promotional_messages, ["CUSTOMER_ID", "CUSTOMER_NAME", "PROMOTIONAL_MESSAGE"])

    print("Removing duplicates from promotional messages...")
    promotional_messages_df = promotional_messages_df.distinct()

    print("Showing the final promotional messages data without duplicates:")
    promotional_messages_df.show(truncate=False)

    print("Saving distinct promotional messages to Gold layer...")
    promotional_messages_df.write \
        .mode("overwrite") \
        .format("delta") \
        .saveAsTable("gold_ec.promotional_messages")

    print("Promotional messages table created in Gold layer.")

process_promotions(spark)



Loading customer segmentation data from Silver layer...
Loading real-time sales data from Bronze layer...
Loading product data from DB2...
Processing High Value Customers...
Generating promotional messages for segment with message: 'Enjoy 20% off on our premium range of products!'
High Value Customers promotion generated.
Processing Medium Value Customers...
Generating promotional messages for segment with message: 'Get a special 10% discount on your next purchase!'
Medium Value Customers promotion generated.
Processing Low Value Customers...
Generating promotional messages for segment with message: 'Welcome back! Enjoy 5% off your next order!'
Low Value Customers promotion generated.
Creating DataFrame for promotional messages...
Removing duplicates from promotional messages...
Showing the final promotional messages data without duplicates:
+-----------+---------------+-------------------------------------------------+
|CUSTOMER_ID|CUSTOMER_NAME  |PROMOTIONAL_MESSAGE                  

In [7]:
from pyspark.sql import functions as F
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

def process_sales_data(spark):
    # Load historical sales data (Parquet format)
    historical_sales = spark.read.format("parquet").load("s3a://sparkdb2-target/bronze_hist_sales/data/")
    print("Historical sales data loaded")

    # Load current sales data (Delta format)
    current_sales = spark.read.format("delta").load("s3a://spark-dl/bronze_ec/realtime_sales_new/")
    print("Current sales data loaded")

    # Preprocessing: Select necessary columns for sales
    historical_sales_selected = historical_sales.select(
        F.col("SALES_ID").cast("long").alias("sales_id"),
        F.col("PRODUCT_ID").cast("long").alias("product_id"),
        F.col("QUANTITY").cast("long").alias("quantity"),
        F.col("TOTAL_AMOUNT").cast("double").alias("total_amount"),
        F.col("SALE_DATE").cast("timestamp").alias("sale_date")
    )

    current_sales_selected = current_sales.select(
        F.col("sales_id").cast("long").alias("sales_id"),
        F.col("product_id").cast("long").alias("product_id"),
        F.col("quantity").cast("long").alias("quantity"),
        F.col("total_amount").cast("double").alias("total_amount"),
        F.col("sale_date").cast("timestamp").alias("sale_date")
    )

    # Fill missing values in historical and current sales data
    historical_sales_selected = historical_sales_selected.fillna({"quantity": 0, "total_amount": 0.0})
    current_sales_selected = current_sales_selected.fillna({"quantity": 0, "total_amount": 0.0})

    # Combine the datasets
    combined_sales = historical_sales_selected.union(current_sales_selected)

    # Aggregate sales data by month (both historical and current)
    combined_sales_monthly = combined_sales.withColumn(
        "sale_month", F.date_trunc('month', F.col('sale_date'))
    ).groupBy(
        'product_id', 'sale_month'
    ).agg(
        F.sum('total_amount').alias('total_sales'),
        F.sum('quantity').alias('total_quantity')
    ).orderBy('sale_month')

    # Fill missing values in the aggregated columns (total_sales, total_quantity)
    combined_sales_monthly = combined_sales_monthly.fillna({"total_sales": 0.0, "total_quantity": 0})

    # Show the aggregated data
    combined_sales_monthly.show(truncate=False)

    # Feature engineering for total_sales prediction: Prepare the features for linear regression (using total_quantity)
    assembler_sales = VectorAssembler(
        inputCols=['total_quantity'],  # Using only quantity to predict sales
        outputCol='features_sales'
    )

    combined_sales_with_features_sales = assembler_sales.transform(combined_sales_monthly)

    # Feature engineering for total_quantity prediction: Prepare the features for linear regression (using total_sales)
    assembler_quantity = VectorAssembler(
        inputCols=['total_sales'],  # Using sales to predict quantity
        outputCol='features_quantity'
    )

    combined_sales_with_features_quantity = assembler_quantity.transform(combined_sales_monthly)

    # Split the data into training and test sets (80% training, 20% testing) for sales prediction
    train_data_sales, test_data_sales = combined_sales_with_features_sales.randomSplit([0.8, 0.2], seed=42)

    # Train a Linear Regression model for predicting total_sales
    lr_sales = LinearRegression(featuresCol='features_sales', labelCol='total_sales')
    lr_model_sales = lr_sales.fit(train_data_sales)

    # Split the data into training and test sets (80% training, 20% testing) for quantity prediction
    train_data_quantity, test_data_quantity = combined_sales_with_features_quantity.randomSplit([0.8, 0.2], seed=42)

    # Train a Linear Regression model for predicting total_quantity
    lr_quantity = LinearRegression(featuresCol='features_quantity', labelCol='total_quantity')
    lr_model_quantity = lr_quantity.fit(train_data_quantity)

    # Evaluate the sales model on test data
    test_results_sales = lr_model_sales.evaluate(test_data_sales)
    print(f"RMSE (Root Mean Squared Error) for sales prediction: {test_results_sales.rootMeanSquaredError}")
    print(f"R² (Coefficient of Determination) for sales prediction: {test_results_sales.r2}")

    # Evaluate the quantity model on test data
    test_results_quantity = lr_model_quantity.evaluate(test_data_quantity)
    print(f"RMSE (Root Mean Squared Error) for quantity prediction: {test_results_quantity.rootMeanSquaredError}")
    print(f"R² (Coefficient of Determination) for quantity prediction: {test_results_quantity.r2}")

    # Predict future sales (for the next month)
    future_months_df = combined_sales_monthly.select("product_id").distinct().crossJoin(
        spark.range(1, 2).select(F.col("id").alias("future_month"))
    ).withColumn(
        "sale_month", F.add_months(F.current_date(), 1)  # Predict for the next month
    )

    # Join future_months_df with combined_sales_monthly to get total_quantity and total_sales
    future_months_with_data = future_months_df.join(
        combined_sales_monthly.withColumnRenamed("sale_month", "historical_sale_month"),
        on='product_id', how='left'
    ).select(
        "product_id", "sale_month", "total_quantity", "total_sales"
    )

    # Fill any null values in the future predictions with 0
    future_months_with_data = future_months_with_data.fillna({"total_quantity": 0, "total_sales": 0.0})

    # Add features for prediction (sales and quantity predictions)
    future_months_with_features_sales = assembler_sales.transform(future_months_with_data)
    future_months_with_features_quantity = assembler_quantity.transform(future_months_with_data)

    # Generate predictions for total_sales
    future_predictions_sales = lr_model_sales.transform(future_months_with_features_sales).withColumnRenamed(
        "prediction", "prediction_sales"
    )

    # Generate predictions for total_quantity
    future_predictions_quantity = lr_model_quantity.transform(future_months_with_features_quantity).withColumnRenamed(
        "prediction", "prediction_quantity"
    )

    # Combine the predictions into a single DataFrame
    future_predictions_combined = future_predictions_sales.join(
        future_predictions_quantity,
        on=["product_id", "sale_month"],
        how="inner"
    )

    # Show the predictions for total_sales and total_quantity
    future_predictions_combined.select("product_id", "sale_month", "prediction_sales", "prediction_quantity").show(truncate=False)

    # Write the combined future prediction output to a Delta table in the Gold layer
    future_predictions_combined.select('product_id', 'sale_month', 'prediction_sales', 'prediction_quantity') \
        .write \
        .mode("overwrite") \
        .format("delta") \
        .saveAsTable("gold_ec.aggregated_sales_and_quantity_predictions")

    print("Aggregated future sales and quantity predictions table created in Gold layer.")

# Call the function to run the processing
process_sales_data(spark)


Historical sales data loaded
Current sales data loaded
+----------+-------------------+------------------+--------------+
|product_id|sale_month         |total_sales       |total_quantity|
+----------+-------------------+------------------+--------------+
|202       |2024-01-01 00:00:00|1249.95           |5             |
|201       |2024-01-01 00:00:00|1499.95           |5             |
|null      |2024-01-01 00:00:00|174.85            |15            |
|null      |2024-02-01 00:00:00|89.85             |15            |
|203       |2024-02-01 00:00:00|999.95            |5             |
|205       |2024-03-01 00:00:00|3499.9500000000003|5             |
|206       |2024-03-01 00:00:00|1999.95           |5             |
|204       |2024-03-01 00:00:00|1749.95           |5             |
|null      |2024-04-01 00:00:00|29.95             |5             |
|208       |2024-04-01 00:00:00|649.95            |5             |
|207       |2024-04-01 00:00:00|1999.9            |10            |
|210   